In [1]:
from IPython.display import HTML, display, Image

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
#get_ipython().events.register('pre_run_cell', set_css)



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Running LLM on localhost Using ngrok

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!pip install aiohttp pyngrok

In [5]:
!ngrok config add-authtoken 2g5PmHB4yCrLM13WqaZgZEb9bqk_23dXJ4mMQiYtymTKQBCbx

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
import threading
import time
import os
import asyncio
from pyngrok import ngrok
import threading
import queue
import time
from threading import Thread

# Get your ngrok token from your ngrok account:
# https://dashboard.ngrok.com/get-started/your-authtoken
token="YOUR_ngrok_TOKEN"
ngrok.set_auth_token(token)

# set up a stoppable thread (not mandatory, but cleaner if you want to stop this later
class StoppableThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def is_stopped(self):
        return self._stop_event.is_set()

def start_ngrok(q, stop_event):
    try:
        # Start an HTTP tunnel on the specified port
        public_url = ngrok.connect(11434)
        # Put the public URL in the queue
        q.put(public_url)
        # Keep the thread alive until stop event is set
        while not stop_event.is_set():
            time.sleep(1)  # Adjust sleep time as needed
    except Exception as e:
        print(f"Error in start_ngrok: {e}")

In [7]:
# Create a queue to share data between threads
url_queue = queue.Queue()

# Start ngrok in a separate thread
ngrok_thread = StoppableThread(target=start_ngrok, args=(url_queue, StoppableThread.is_stopped))
ngrok_thread.start()

In [8]:
# Wait for the ngrok tunnel to be established
while True:
    try:
        public_url = url_queue.get()
        if public_url:
            break
        print("Waiting for ngrok URL...")
        time.sleep(1)
    except Exception as e:
        print(f"Error in retrieving ngrok URL: {e}")

print("Ngrok tunnel established at:", public_url)

Error in start_ngrok: 'function' object has no attribute 'is_set'
Ngrok tunnel established at: NgrokTunnel: "https://feac-35-236-134-192.ngrok-free.app" -> "http://localhost:11434"


In [9]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [10]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


In [11]:
!ollama pull llama3:8b

2024/05/25 12:23:59 routes.go:1008: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:*] OLLAMA_RUNNERS_DIR: OLLAMA_TMPDIR:]"
time=2024-05-25T12:23:59.275Z level=INFO source=images.go:704 msg="total blobs: 5"
time=2024-05-25T12:23:59.278Z level=INFO source=images.go:711 msg="total unused blobs removed: 0"
time=2024-05-25T12:23:59.279Z level=INFO source=routes.go:1054 msg="Listening on 127.0.0.1:11434 (version 0.1.38)"
time=2024-05-25T12:23:59.281Z level=INFO source=payload.go:30 msg="extracting embedded files" dir=/tmp/ollama1163257866/runners
time=2024-05-25T12:24:13.957Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cp

In [12]:
# !ollama pull llama3:70b

In [13]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [14]:
!curl http://localhost:11434/api/chat -d '{ "model": "llama3:8b", "stream": false, "messages": [{ "role": "user", "content": "Why is the sky green?" }]}'

time=2024-05-25T12:24:57.564Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=19 memory.available="3.2 GiB" memory.required.full="4.6 GiB" memory.required.partial="3.1 GiB" memory.required.kv="256.0 MiB" memory.weights.total="4.1 GiB" memory.weights.repeating="3.7 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="164.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-05-25T12:24:57.564Z level=INFO source=server.go:320 msg="starting llama server" cmd="/tmp/ollama1163257866/runners/cpu_avx2/ollama_llama_server --model /root/.ollama/models/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa --ctx-size 2048 --batch-size 512 --embedding --log-disable --parallel 1 --port 35243"
time=2024-05-25T12:24:57.565Z level=INFO source=sched.go:338 msg="loaded runners" count=1
time=2024-05-25T12:24:57.565Z level=INFO source=server.go:504 msg="waiting for llama runner to start responding"
time=2024-05-25T12:24:57.566Z level=INFO

# Prerequisites

In [15]:
%%capture --no-stderr
!pip install -U langgraph langchain-community langchain-anthropic tavily-python pandas langchain_openai

In [16]:
!pip install --upgrade langchain

In [20]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.

In [17]:
import getpass
import os


# Recommended
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Persian Customer Support Bot"
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "YOUR_LANGCHAIN_API_KEY"
os.environ['TAVILY_API_KEY'] = "TAVILY_API_KEY"

# main

In [18]:
import sys
import os
sys.path.append("/content/drive/MyDrive/GenAI/assignment/assignment-1/")

In [21]:
import os
from data.data_base import load_travel_db
from agent.zeroshot_agent import ZeroShotAgent
from agent.conditional_interrupt import ConditionalInterruptAgent
from agent.add_confirmation import AddConfirmationAgent

local_llm = "llama3:8b"

data_dir = "/content/drive/MyDrive/GenAI/assignment/assignment-1/data"
travel_db = load_travel_db(data_dir)
# agent = ZeroShotAgent(local_llm, travel_db, data_dir)
agent = AddConfirmationAgent(local_llm, travel_db, data_dir)
agent.define_graph()
#agent.run_example()

In [22]:
import uuid

passenger_id = '3442 587242'
config = {
    'configurable': {
        'passenger_id': passenger_id,
        'thread_id': str(uuid.uuid4()),
    }
}

In [23]:
query = "ساعت چنده"
agent.run(query, config)

================================ Human Message =================================

ساعت چنده
[GIN] 2024/05/25 - 12:38:38 | 200 |         4m37s |       127.0.0.1 | POST     "/v1/chat/completions"
================================== Ai Message ==================================

با بررسی تیکت، مشاهده می‌کنیم که ساعت کنونی ۱۲:۳۳:۴۸ است. سند پرواز هم در حال طی شدن است و پرواز LX0112 با هدف رسیدن به فرودگاه بسل در تاریخ ۲۵۵/۴-۲۰۲۴ ساعت ۱۴:۰۶:۴۷ برنامه ریزی شده‌است.


In [24]:
query = "دمای هوای تهران چقدره"
agent.run(query, config)

================================ Human Message =================================

دمای هوای تهران چقدره
[GIN] 2024/05/25 - 12:41:34 | 200 |         2m49s |       127.0.0.1 | POST     "/v1/chat/completions"
================================== Ai Message ==================================

متاسفانه، در لحظه جاری، من به عنوان کارشناس پشتیبانی، اطلاعات هواشناسی واقعی تهران را ندارم. ولی می‌توانم بگویم که دمای هوای تهران معمولاً در ماه می-dependent بر عواملی چون زمان روز، فصل و شرایط اقلیم تغییر می‌کند. برای کسب اطلاعات دقیق تر،Recommendation می‌کنم به سایت رسمی هواشناسی ایران مراجعه کنید یا یک اپلیکیشن هواشناسی معتبر را دانلود کنید.


In [25]:
query = "حداکثر برای چند مسافر میشه بلیط قطار تهیه کرد؟"
agent.run(query, config)

================================ Human Message =================================

حداکثر برای چند مسافر میشه بلیط قطار تهیه کرد؟
[GIN] 2024/05/25 - 12:45:15 | 200 |         3m36s |       127.0.0.1 | POST     "/v1/chat/completions"
================================== Ai Message ==================================

پاسخ این سوال از ردمت پرواز و انتخاب آهنری وابسته است! بر اساس مقررات شرکت قطارهای AliBaba، حداکثر تعداد مسافر که می‌توانند به صورت یکجا در یک رزرو وارد شوند، ۹ نفر است. اما برای سفرهای بین المللی یا سفرهای خاص دیگر، ممکن است این عدد تغییر کند. بهتر است قبل از تهیه بلیط، با بخش پشتیبانی قطار تماس بگیرید و اطلاعات لازم را حاصل کنید.


In [26]:
agent.run("سلام. ساعت پرواز من کی است؟", config)

================================ Human Message =================================

سلام. ساعت پرواز من کی است؟
[GIN] 2024/05/25 - 12:46:48 | 200 |         1m27s |       127.0.0.1 | POST     "/v1/chat/completions"
================================== Ai Message ==================================

سلام! بر اساس اطلاعات تیکت، پروازم شما پرواز LX0112 به فرودگاه بسل در تاریخ ۲۵۵/۴-۲۰۲۴ ساعت ۱۴:۰۶:۴۷ برنامه ریزی شده‌است.


In [27]:
question = "چه پروازهایی به مقصد BSL وجود داره؟"
agent.run(question, config)

================================ Human Message =================================

چه پروازهایی به مقصد BSL وجود داره؟
[GIN] 2024/05/25 - 12:49:42 | 200 |         2m47s |       127.0.0.1 | POST     "/v1/chat/completions"
================================== Ai Message ==================================

بر اساس سیستم رزرو آلبABA، پرواز LX0112 به فرودگاه بسل در سیستمی از پروازهای ما قرار دارد. علاوه بر این، می‌توانم بگویم که دیگر پروازها نیز به مقصد بسل وجود دارند، اما برای مشاهده ی کامل فهرست پروازها، پیشنهاد می‌کنم که به صفحه اصلی رزرو آلبABA مراجعه کنید و گزینش‌هایتان را انجام دهید.


In [ ]:
local_llm = "llama3:8b"
zsh_agent = ZeroShotAgent(local_llm, travel_db, data_dir)
zsh_agent.define_graph()
zsh_agent.run_example()

================================ Human Message =================================

سلام. ساعت پرواز چند است؟
time=2024-05-20T17:32:46.430Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=33 memory.available="14.6 GiB" memory.required.full="5.0 GiB" memory.required.partial="5.0 GiB" memory.required.kv="256.0 MiB" memory.weights.total="4.1 GiB" memory.weights.repeating="3.7 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="164.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-05-20T17:32:46.435Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=33 memory.available="14.6 GiB" memory.required.full="5.0 GiB" memory.required.partial="5.0 GiB" memory.required.kv="256.0 MiB" memory.weights.total="4.1 GiB" memory.weights.repeating="3.7 GiB" memory.weights.nonrepeating="411.0 MiB" memory.graph.full="164.0 MiB" memory.graph.partial="677.5 MiB"
time=2024-05-20T17:32:46.444Z level=INFO source=server.go:320 msg="sta